# 载入数据

In [1]:
import tensorflow as tf
#导入Tensorflow提供的读取MNIST的模块
import tensorflow.examples.tutorials.mnist.input_data as input_data
#读取MNIST数据
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


# 建立模型
#### 定义标签数据占位符

In [2]:
x = tf.placeholder(tf.float32, [None, 784], name="X")
y = tf.placeholder(tf.float32, [None, 10], name="Y")

# 构建模型

In [3]:
H1_NN = 256 # 第1隐藏层神经元为256
H2_NN = 64  # 第2隐藏层神经元为64
H3_NN = 32  # 第2隐藏层神经元为32

In [4]:
# 输入层 - 第1隐藏层参数和偏置项
W1 = tf.Variable(tf.truncated_normal([784, H1_NN], stddev=0.1))
b1 = tf.Variable(tf.zeros([H1_NN]))

# 第1隐藏层 - 第2隐藏层参数和偏置项
W2 = tf.Variable(tf.truncated_normal([H1_NN, H2_NN], stddev=0.1))
b2 = tf.Variable(tf.zeros([H2_NN]))

# 第2隐藏层 - 第3隐藏层参数和偏置项
W3 = tf.Variable(tf.truncated_normal([H2_NN, H3_NN], stddev=0.1))
b3 = tf.Variable(tf.zeros([H3_NN]))

# 第3隐藏层 - 输出层参数和偏置项
W4 = tf.Variable(tf.truncated_normal([H3_NN, 10], stddev=0.1))
b4 = tf.Variable(tf.zeros([10]))

In [5]:
# 计算第1隐藏层结果
Y1 = tf.nn.relu(tf.matmul(x, W1) + b1)

# 计算第2隐藏层结果
Y2 = tf.nn.relu(tf.matmul(Y1, W2) + b2)

# 计算第3隐藏层结果
Y3 = tf.nn.relu(tf.matmul(Y2, W3) + b3)

# 计算输出结果
forward = tf.matmul(Y3, W4) + b4
pred = tf.nn.softmax(forward)

# 训练模型

### 定义损失函数

In [6]:
#交叉熵
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=forward, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 设置训练参数

In [7]:
train_epochs = 40
batch_size = 50
total_batch = int(mnist.train.num_examples/batch_size)
display_step = 1
learning_rate = 0.01

### 选择优化器

In [8]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

### 定义准确率

In [9]:
correct_prediction = tf.equal(tf.argmax(y, 1),tf.argmax(pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### 训练模型

In [10]:
#记录训练开始时间
from time import time
startTime = time()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for epoch in range(train_epochs):
    for batch in range(total_batch):
        xs, ys = mnist.train.next_batch(batch_size)
        sess.run(optimizer,feed_dict={x: xs, y: ys})
    #total_batch个批次训练完成后，使用验证数据计算误差与准确率
    loss, acc = sess.run([loss_function, accuracy],feed_dict={x: mnist.validation.images,
                                                              y: mnist.validation.labels})
    if(epoch+1) % display_step ==0:
        print("Train Epoch:","%02d" % (epoch+1),
              "Loss=","{:.9f}".format(loss), " Accuracy=","{:.4f}".format(acc))
#运行总时间
duration = time()-startTime
print("Train Finished takes:","{:.2f}".format(duration))

Train Epoch: 01 Loss= 0.140489668  Accuracy= 0.9596
Train Epoch: 02 Loss= 0.164851874  Accuracy= 0.9560
Train Epoch: 03 Loss= 0.130485445  Accuracy= 0.9728
Train Epoch: 04 Loss= 0.149478376  Accuracy= 0.9692
Train Epoch: 05 Loss= 0.179703116  Accuracy= 0.9596
Train Epoch: 06 Loss= 0.137343451  Accuracy= 0.9670
Train Epoch: 07 Loss= 0.140299484  Accuracy= 0.9686
Train Epoch: 08 Loss= 0.132477909  Accuracy= 0.9742
Train Epoch: 09 Loss= 0.126109242  Accuracy= 0.9748
Train Epoch: 10 Loss= 0.157294124  Accuracy= 0.9680
Train Epoch: 11 Loss= 0.139471918  Accuracy= 0.9710
Train Epoch: 12 Loss= 0.204612404  Accuracy= 0.9708
Train Epoch: 13 Loss= 0.205232099  Accuracy= 0.9718
Train Epoch: 14 Loss= 0.141823843  Accuracy= 0.9738
Train Epoch: 15 Loss= 0.188388675  Accuracy= 0.9708
Train Epoch: 16 Loss= 0.172192827  Accuracy= 0.9740
Train Epoch: 17 Loss= 0.178750515  Accuracy= 0.9728
Train Epoch: 18 Loss= 0.190618068  Accuracy= 0.9716
Train Epoch: 19 Loss= 0.210707009  Accuracy= 0.9704
Train Epoch:

# 评估模型

In [11]:
accu_test = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})
print("Test Accuracy:", accu_test)

Test Accuracy: 0.971


# 进行预测

In [12]:
#由于pred预测结果是one-hot编码格式，所以需要转换为0~9数字。
prediction_result=sess.run(tf.argmax(pred,1), feed_dict={x: mnist.test.images})

In [13]:
#查看预测结果中的前10项
prediction_result[0:10]

array([7, 2, 1, 0, 4, 1, 4, 9, 5, 9])